<a href="https://colab.research.google.com/github/return-z/OneShotLearning/blob/main/oneshotlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip

!unzip images_background.zip
!unzip images_evaluation.zip


--2024-01-03 19:31:12--  https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_background.zip [following]
--2024-01-03 19:31:13--  https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_background.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9464212 (9.0M) [application/zip]
Saving to: ‘images_background.zip.1’

images_background.z 100%[===================>]   9.03M  --.-KB/s    in 0.06s   

2024-01-03 19:31:13 (154 MB/s) - ‘images_background.zip.1’ saved

In [ ]:
import os
import cv2
import numpy as np

def load_images(path, size=(105, 105)):
    images, labels = [], []
    for alphabet in os.listdir(path):
        for character in os.listdir(os.path.join(path, alphabet)):
            for img_file in os.listdir(os.path.join(path, alphabet, character)):
                img_path = os.path.join(path, alphabet, character, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, size)
                images.append(img)
                labels.append(character)
    return np.array(images), np.array(labels)


In [ ]:
def preprocess_images(images):
    images = images.astype('float32') / 255.0  # Normalize
    images = np.expand_dims(images, axis=-1)   # Reshape for CNN input
    return images

# Load and preprocess the images
train_images, train_labels = load_images('images_background')
train_images = preprocess_images(train_images)

test_images, test_labels = load_images('images_evaluation')
test_images = preprocess_images(test_images)


In [ ]:
import random

def make_pairs(images, labels):
    pair_images, pair_labels = [], []
    label_dict = dict()

    for idx, label in enumerate(labels):
        label_dict.setdefault(label, []).append(images[idx])

    unique_labels = np.unique(labels)
    num_classes = len(unique_labels)

    for idx, label in enumerate(unique_labels):
        for _ in range(len(label_dict[label])):
            # Create a pair of similar images
            similar_image_idx = random.choice(label_dict[label])
            pair_images.append([images[idx], similar_image_idx])
            pair_labels.append([1])

            # Create a pair of dissimilar images
            different_label = random.choice(list(set(unique_labels) - set([label])))
            different_image_idx = random.choice(label_dict[different_label])
            pair_images.append([images[idx], different_image_idx])
            pair_labels.append([0])

    return np.array(pair_images), np.array(pair_labels)

# Create pairs for training and testing
train_pairs, train_pair_labels = make_pairs(train_images, train_labels)
test_pairs, test_pair_labels = make_pairs(test_images, test_labels)


In [ ]:
train_pairs = train_pairs.astype('float32')
test_pairs = test_pairs.astype('float32')

# Reshape for CNN input
train_pairs = train_pairs.reshape(-1, 2, 105, 105, 1)
test_pairs = test_pairs.reshape(-1, 2, 105, 105, 1)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras import backend as K

# Define the base network architecture
def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

# Input shape
input_shape = train_pairs[0][0].shape

# Create the base network
base_network = create_base_network(input_shape)

# Create the inputs for the two branches of the Siamese network
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# Because we re-use the same instance 'base_network',
# the weights of the network will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate the L1 distance between the two branch outputs
distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([processed_a, processed_b])

# Add a dense layer with a sigmoid unit to generate the similarity score
output = Dense(1, activation='sigmoid')(distance)

model = Model(inputs=[input_a, input_b], outputs=output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Split the pairs and labels for training
pair1 = train_pairs[:, 0]
pair2 = train_pairs[:, 1]
labels = train_pair_labels

# Train the model
history = model.fit([pair1, pair2], labels, epochs=20, validation_split=0.2)


NameError: ignored